# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [14]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [15]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [16]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [17]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

The citations file is read and stored. It is delimited by ',' such that the first column is Citing and the
second column is Cited. 

In [18]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



The first 5 lines of the citations table is outputted. 

In [19]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

The patents file is read and delimitted by ','. 

In [20]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

The first 5 lines of the patents table is output. 

In [21]:
naturalJoin = citations.join(patents, citations.CITED == patents.PATENT)
# naturalJoin.show(10)
cited_join = naturalJoin.select("CITING","CITED","POSTATE")
cited_join_renamed = cited_join.withColumnRenamed("CITING","CITINGA").withColumnRenamed("CITED","CITEDA").withColumnRenamed("POSTATE","POSTATEA")






A join is performed between the citations and the patents file based on the cited id.
Out of the table only the citing id, cited id, and the cited_state is chosen. The columns that are chosen 
have been renamed. 

In [22]:
cited_join_renamed.show(10)

+-------+-------+--------+
|CITINGA| CITEDA|POSTATEA|
+-------+-------+--------+
|3951073|3071083|    null|
|4401034|3071083|    null|
|4802416|3071083|    null|
|4045189|3071452|    null|
|4192656|3071452|    null|
|3949375|3071753|      MN|
|4120573|3071753|      MN|
|4271479|3071753|      MN|
|4345315|3071753|      MN|
|4496943|3071753|      MN|
+-------+-------+--------+
only showing top 10 rows



In [23]:
fullJoin = cited_join_renamed.join(patents, cited_join_renamed.CITINGA == patents.PATENT)

fullJoinSubset = fullJoin.select("CITINGA","CITEDA","POSTATEA","POSTATE")
fullJoinRenamed = fullJoinSubset.withColumnRenamed("CITINGA","CITING").withColumnRenamed("CITEDA","CITED").withColumnRenamed("POSTATEA","CITED_STATE").withColumnRenamed("POSTATE","CITING_STATE")
fullJoinRenamed.show(10)

+-------+-------+-----------+------------+
| CITING|  CITED|CITED_STATE|CITING_STATE|
+-------+-------+-----------+------------+
|3858527|3638586|         CA|        null|
|3858527|3699902|         OH|        null|
|3858527|3602157|         TX|        null|
|3858597|3815160|         NY|          MT|
|3858597|3675252|         AZ|          MT|
|3859029|3741706|         OH|          NY|
|3859029|3685936|         OH|          NY|
|3859627|3368197|         MI|          MI|
|3859627|3226674|       null|          MI|
|3859627|3359539|         CA|          MI|
+-------+-------+-----------+------------+
only showing top 10 rows



A second join is performed between (citing_id, cited_id and the cited_state ) table  and the patent table. It outputs the citing_id, cited_id, cited_state and the citing_state. 

In [24]:
fullJ = fullJoinRenamed.filter('CITED_STATE == CITING_STATE').groupBy("CITING").count()
fullJ.show(5)

+-------+-----+
| CITING|count|
+-------+-----+
|4240165|    3|
|5096364|    2|
|5122917|    5|
|5203482|    2|
|5583013|    9|
+-------+-----+
only showing top 5 rows



All the rows are filtered on the basis of cited and the citing states being equal. They are then grouped by the citing_id to find out the count of same state citations per patent_id. 

In [25]:
finaltable = patents.join(fullJ, fullJ.CITING == patents.PATENT)
finaltable.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
|3859627| 1975| 5485|   1972|     US|     MI|  332120|      2|    14|   180|  5|    55|    7|       2|     1.0|    0.5|  0.4082|    12.5|  6.8571|     0.0|     0.0|     0.0|     0.0|3859627|    1|
|3860191| 1975| 5492|   1972|     US|     CA|  452885|      2|     5|   242|  5|    51|    4|       4|    0.75|    0.0|  0.4444|     5.0|    11.0|     0.0|     0.0|     0.0|     0.0|3860191|    1|
|3861180| 1975|

The count table is then joined with the patent table such that the count is appended at the end of the patent information for each patent ID. 

In [26]:
result = finaltable.drop(col("CITING"))
result.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|3859627| 1975| 5485|   1972|     US|     MI|  332120|      2|    14|   180|  5|    55|    7|       2|     1.0|    0.5|  0.4082|    12.5|  6.8571|     0.0|     0.0|     0.0|     0.0|    1|
|3860191| 1975| 5492|   1972|     US|     CA|  452885|      2|     5|   242|  5|    51|    4|       4|    0.75|    0.0|  0.4444|     5.0|    11.0|     0.0|     0.0|     0.0|     0.0|    1|
|3861180| 1975| 5499|   1973|     US|     AZ|    null| 

Duplicated column citing is removed. 

In [29]:
result.filter(result.PATENT == "6009554").show()

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|6009554| 1999|14606|   1997|     US|     NY|  219390|      2|  null|   714|  2|    22|    9|       0|     1.0|   null|    null|    null| 12.7778|  0.1111|  0.1111|    null|    null|    8|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+



Validating for patent id = 6009554 

In [28]:
Top_10_Same_State_Count = result.orderBy("count", ascending=False).show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|  125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|  103|
|6008204| 1999|14606|   1998|     US|     CA|  749584| 

Top 10 same state counts is taken from the result of the previous table and displayed. 